# Cifar-10 test for image classification

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download EfficientNet (v.2, small model) with Imagenet weights (1000 classes)

In [5]:
enet = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    include_preprocessing=True
)

87549136/87549136 [==============================] - 1s 0us/step


In [6]:
enet.summary()

Model: "efficientnetv2-s"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 384, 384, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 384, 384, 3)          0         ['input_1[0][0]']             
                                                                                                  
 stem_conv (Conv2D)          (None, 192, 192, 24)         648       ['rescaling[0][0]']           
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 192, 192, 24)         96        ['stem_conv[0][0]']           
 n)                                                                                

Make all layers untrainable except for the very last convolutional layer

In [7]:
for layer in enet.layers:
    layer.trainable = False
enet.layers[-6].trainable  = True

Cifar-10 testing

In [8]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [9]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Lambda layer for preprocessing

In [10]:
def resize(x):
    return tf.image.resize(x,size=(384,384),method='bilinear')

Modify the model

In [11]:
input_shape = (32,32,3)

In [12]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = enet(prep)
enet_mod = Model(inputs=input_layer, outputs=out)

In [13]:
enet_mod.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-10 data and then evaluate (this will be efficient enough if trained on the complete dataset ...)

In [14]:
enet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 225s 130ms/step - loss: 0.4087 - top_1_categorical_accuracy: 0.8694


In [15]:
enet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 34s 99ms/step - loss: 0.2648 - top_1_categorical_accuracy: 0.9068


[0.2647807002067566, 0.9067999720573425]

Try the same with adding a Cerebros "add-on" network

In [16]:
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [10]

Use some 15k random samples from Cifar-10 to speed up the process

In [35]:
num_samples = 15_000
rng = np.random.default_rng()
ind = rng.permutation(X_train.shape[0])[:num_samples]

In [36]:
training_x   = [tf.constant(X_train[ind,:,:,:])]
y_train_cat  = to_categorical(y_train[ind], 10)
train_labels = [tf.constant(y_train_cat)]

In [37]:
enet = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    include_preprocessing=True
)

In [38]:
for layer in enet.layers:
    layer.trainable = False
enet.layers[-6].trainable = True

In [39]:
enet_io = Model(inputs=enet.layers[0].input,
                outputs=enet.layers[-3].output)

In [40]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = Flatten()(enet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [41]:
activation = 'swish'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 5  # [1, 100]
batch_size = 32
maximum_levels = 4  # [3,7]
maximum_units_per_level = 7  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [42]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar10_efficientnet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [43]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'10': <class 'cerebros.units.units.FinalDenseUnit'>}]
3
Trying to create Final level 3
Trying to create level 3
We think level final level 3's predecessors are: [0, 1, 2]
levels:
[0, 1, 2, 3]
{'0': 'InputUnitModule'}
InputLevel.input_shapes [(32, 32, 3)]
{'3': <class 'cerebros.units.units.DenseUnit'>}
{'3': <class 'cerebros.units.units.DenseUnit'>}
{'2': <class 'cereb

In [44]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.9203333258628844
top-1 categorical accuracy


Evaluating the best model found

In [45]:
best_model_found = cerebros_automl.get_best_model()

In [46]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [47]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_1_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_1_InputLevel                                                                       
 _0000000000000000_tr_1_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _1_0_inp (InputLayer)                                                                            
                                                                                                  
 model_4 (Functional)        (None, 1280)                 2033136   ['NeuralNetworkFuture_00000000
                                          

In [48]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 10)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 38s 105ms/step - loss: 0.5036 - eval_top_1_categorical_accuracy: 0.9061 - eval_top_5_categorical_accuracy: 0.9927


[0.5036455988883972, 0.9060999751091003, 0.9926999807357788]